# Frank Asket's Week 2 Exercise

[Frank Asket](https://github.com/frank-asket) — *Founder & CTO building Human-Centered AI infrastructure.*

Full prototype of the **technical Q&A** from Week 1: **Gradio UI**, **streaming**, **system prompt** for expertise, **model switching** (OpenRouter GPT vs Ollama Llama), and a **tool** (current time) so the assistant can answer “What time is it?”.

In [1]:
# imports

import os
import json
from datetime import datetime, timezone
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# environment & API client (OpenRouter preferred, fallback OpenAI)

load_dotenv(override=True)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

if openrouter_api_key and openrouter_api_key.startswith("sk-or-"):
    openai = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
    MODEL_GPT = "openai/gpt-4o-mini"
    print("Using OpenRouter.")
elif openai_api_key:
    openai = OpenAI(api_key=openai_api_key)
    MODEL_GPT = "gpt-4o-mini"
    print("Using OpenAI.")
else:
    openai = OpenAI()
    MODEL_GPT = "gpt-4o-mini"
    print("Using default client (set OPENROUTER_API_KEY or OPENAI_API_KEY in .env).")

MODEL_OLLAMA = "llama3.2:3b-instruct-q4_0"
OLLAMA_BASE = "http://localhost:11434/v1"

Using OpenRouter.


In [3]:
# system prompts (expertise personas)

SYSTEM_PROMPTS = {
    "Technical tutor": (
        "You are a helpful technical tutor. Answer questions about Python, software engineering, "
        "data science and LLMs. Use markdown and be clear. If the user asks for the current time, "
        "use the get_current_time tool."
    ),
    "Code reviewer": (
        "You are a senior code reviewer. Explain code snippets, suggest improvements, and point out "
        "pitfalls. Use markdown. If the user asks what time it is, use the get_current_time tool."
    ),
    "LLM explainer": (
        "You explain how LLMs, APIs and prompt engineering work. Be precise and educational. "
        "Use markdown. If the user asks for the current time, use the get_current_time tool."
    ),
}

In [4]:
# tool: get current time (demonstrates tool use)

def get_current_time(timezone_name: str = "UTC") -> str:
    """Return the current date and time in the given timezone (e.g. UTC, Europe/Paris)."""
    try:
        from zoneinfo import ZoneInfo
        tz = ZoneInfo(timezone_name)
    except Exception:
        tz = timezone.utc
    now = datetime.now(tz)
    return f"Current time in {timezone_name}: {now.strftime('%Y-%m-%d %H:%M:%S %Z')}"

time_tool = {
    "name": "get_current_time",
    "description": "Get the current date and time in a given timezone (e.g. UTC, Europe/Paris, America/New_York).",
    "parameters": {
        "type": "object",
        "properties": {
            "timezone_name": {
                "type": "string",
                "description": "IANA timezone name, e.g. UTC or Europe/Paris",
                "default": "UTC"
            },
        },
        "required": [],
        "additionalProperties": False
    }
}
TOOLS = [{"type": "function", "function": time_tool}]

In [5]:
def strip_code_fence(text: str) -> str:
    """Remove code-fence wrappers so markdown displays cleanly."""
    if not text or not text.strip():
        return text
    s = text
    if s.startswith("```markdown"):
        i = s.find("\n")
        s = s[i + 1:] if i != -1 else s[11:]
    elif s.startswith("```"):
        i = s.find("\n")
        s = s[i + 1:] if i != -1 else s[3:]
    if s.rstrip().endswith("```"):
        s = s[:s.rstrip().rfind("```")].rstrip()
    return s


def handle_tool_calls(message):
    """Execute tool calls and return a list of tool results for the API."""
    results = []
    for tc in message.tool_calls:
        name = tc.function.name
        args = json.loads(tc.function.arguments) if tc.function.arguments else {}
        if name == "get_current_time":
            out = get_current_time(**args)
        else:
            out = f"Unknown tool: {name}"
        results.append({"role": "tool", "content": out, "tool_call_id": tc.id})
    return results

In [6]:
def chat_stream(message, history, model_choice, persona_key):
    """Streaming chat: supports GPT (OpenRouter/OpenAI) with optional tool, or Ollama. Yields cumulative response for Gradio."""
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    system = SYSTEM_PROMPTS.get(persona_key, list(SYSTEM_PROMPTS.values())[0])
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]

    use_ollama = "Ollama" in model_choice
    client = openai
    model = MODEL_GPT
    use_tools = False
    if use_ollama:
        try:
            client = OpenAI(base_url=OLLAMA_BASE, api_key="ollama")
            model = MODEL_OLLAMA
        except Exception:
            yield "Ollama not available. Start with: `ollama serve` and `ollama pull llama3.2`"
            return
    else:
        use_tools = True

    # Tool loop for GPT (one round of tool calls, then yield final answer)
    if use_tools:
        response = client.chat.completions.create(model=model, messages=messages, tools=TOOLS)
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            messages.append(msg)
            messages.extend(handle_tool_calls(msg))
            response = client.chat.completions.create(model=model, messages=messages, tools=TOOLS)
        final_content = response.choices[0].message.content or ""
        yield strip_code_fence(final_content)
        return

    # Ollama: stream chunks
    stream = client.chat.completions.create(model=model, messages=messages, stream=True)
    acc = ""
    for chunk in stream:
        acc += chunk.choices[0].delta.content or ""
        yield strip_code_fence(acc)

In [9]:
# Gradio UI: model switch, persona (system prompt), streaming chat

with gr.Blocks() as demo:
    gr.Markdown(
        """
        ## Technical Q&A prototype (Week 2)
        - **Model:** OpenRouter GPT or Ollama Llama
        - **Persona:** system prompt sets expertise (tutor, code reviewer, LLM explainer)
        - **Streaming** answers; **tool:** ask *What time is it?* to see the assistant use the clock.
        """
    )
    with gr.Row():
        model_choice = gr.Dropdown(
            ["OpenRouter GPT", "Ollama Llama"],
            value="OpenRouter GPT",
            label="Model"
        )
        persona = gr.Dropdown(
            list(SYSTEM_PROMPTS.keys()),
            value="Technical tutor",
            label="Persona"
        )
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(placeholder="Ask a technical question or 'What time is it?'", label="Message")
    send = gr.Button("Send", variant="primary")
    clear = gr.Button("Clear")

    def respond(message, history, model, persona_name):
        if not message or not message.strip():
            return "", history
        history = history + [{"role": "user", "content": message}]
        full = ""
        for chunk in chat_stream(message, history[:-1], model, persona_name):
            full = chunk
        history = history + [{"role": "assistant", "content": full}]
        return "", history

    msg.submit(respond, [msg, chatbot, model_choice, persona], [msg, chatbot])
    send.click(respond, [msg, chatbot, model_choice, persona], [msg, chatbot])
    clear.click(lambda: [], None, chatbot, queue=False)

demo.launch(inbrowser=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
